[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aurelio-labs/langchain-course/blob/main/chapters/05-agents-intro.ipynb)

#### LangChain Essentials Course

# LangChain Agents Intro

LangChain is one of the most popular open source libraries for AI Engineers. It's goal is to abstract away the complexity in building AI software, provide easy-to-use building blocks, and make it easier when switching between AI service providers.

In this example, we will introduce LangChain's Agents, adding the ability to use tools such as search and calculators to complete tasks that normal LLMs cannot fufil. In this example we will be using OpenAI's `gpt-4o-mini`.

In [3]:
!pip install -qU \
  langchain-core==0.3.33 \
  langchain-openai==0.3 \
  langchain-groq==0.2.0 \
  langchain-community==0.3.16 \
  langsmith==0.3.4 \
  google-search-results==2.4.2

---

> ⚠️ We will be using OpenAI for this example allowing us to run everything via API. If you would like to use Ollama instead, check out the [Ollama LangChain Course](https://github.com/aurelio-labs/langchain-course/tree/main/notebooks/ollama).

---

---

> ⚠️ If using LangSmith, add your API key below:

In [4]:
import os
from getpass import getpass

os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY") or \
    getpass("Enter LangSmith API Key: ")

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "default"

Enter LangSmith API Key: ··········


---

## Introduction to Tools

Tools are a way augment our LLMs with code execution. A tool is simply a function formatted so that our agent can undertstand how to use it, and then execute it. Let's start by creating a few simple tools.

We can use the `@tool` decorator to create an LLM-compatible tool from a standard python function — this function should include a few things for optimal performance:

* A docstring describing what the tool does and when it should be used, this will be read by our LLM/agent and used to decide when to use the tool, and also how to use the tool.

* Clear parameter names that ideally tell the LLM what each parameter is, if it isn't clear we make sure the docstring explains what the parameter is for and how to use it.

* Both parameter and return type annotations.

In [4]:
from langchain_core.tools import tool

@tool
def add(x: float, y: float) -> float:
    """Add 'x' and 'y'."""
    return x + y

@tool
def multiply(x: float, y: float) -> float:
    """Multiply 'x' and 'y'."""
    return x * y

@tool
def exponentiate(x: float, y: float) -> float:
    """Raise 'x' to the power of 'y'."""
    return x ** y

@tool
def subtract(x: float, y: float) -> float:
    """Subtract 'x' from 'y'."""
    return y - x

With the `@tool` decorator our function is turned into a `StructuredTool` object, which we can see below:

In [5]:
add

StructuredTool(name='add', description="Add 'x' and 'y'.", args_schema=<class 'langchain_core.utils.pydantic.add'>, func=<function add at 0x7ff040e3bec0>)

We can see the tool name, description, and arg schema:

In [7]:
print(f"{add.name=}\n{add.description=}")

add.name='add'
add.description="Add 'x' and 'y'."


In [8]:
add.args_schema.model_json_schema()

{'description': "Add 'x' and 'y'.",
 'properties': {'x': {'title': 'X', 'type': 'number'},
  'y': {'title': 'Y', 'type': 'number'}},
 'required': ['x', 'y'],
 'title': 'add',
 'type': 'object'}

In [ ]:
exponentiate.args_schema.model_json_schema()

{'description': "Raise 'x' to the power of 'y'.",
 'properties': {'x': {'title': 'X', 'type': 'number'},
  'y': {'title': 'Y', 'type': 'number'}},
 'required': ['x', 'y'],
 'title': 'exponentiate',
 'type': 'object'}

When invoking the tool, a JSON string output by the LLM will be parsed into JSON and then consumed as kwargs, similar to the below:

In [6]:
import json

llm_output_string = "{\"x\": 5, \"y\": 2}"  # this is the output from the LLM
llm_output_dict = json.loads(llm_output_string)  # load as dictionary
llm_output_dict

{'x': 5, 'y': 2}

This is then passed into the tool function as `kwargs` (keyword arguments) as indicated by the `**` operator - the `**` operator is used to unpack the dictionary into keyword arguments.

In [7]:
exponentiate.func(**llm_output_dict)

25

This covers the basics of tools and how they work, let's move on to creating the agent itself.

## Creating an Agent

We're going to construct a simple tool calling agent. We will use **L**ang**C**hain **E**pression **L**anguage (LCEL) to construct the agent. We will cover LCEL more in the next chapter, but for now - all we need to know is that our agent will be constructed using syntax and components like so:


```
agent = (
    <input parameters, including chat history and user query>
    | <prompt>
    | <LLM with tools>
)
```

We need this agent to remember previous interactions within the conversation. To do that, we will use the `ChatPromptTemplate` with a system message, a placeholder for our chat history, a placeholder for the user query, and finally a placeholder for the agent scratchpad.

The agent scratchpad is where the agent will write it's _"notes"_ as it is working through multiple internal thought and tool-use steps to produce a final output to the user.

In [8]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

Next, we must define our LLM, we will use the `gpt-4o-mini` model with a `temperature` of `0.0`.

In [1]:
# Install required packages
!pip install langchain-groq==0.2.0 langchain-core==0.3.0 pydantic==2.7.0

# Import modules
from langchain_groq import ChatGroq
from langchain_core.caches import BaseCache
import os

# Rebuild the model to resolve Pydantic issues
ChatGroq.model_rebuild()

# Set your Groq API key (replace with your actual key)
os.environ["GROQ_API_KEY"] = "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"

# Instantiate ChatGroq
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2
)

When creating an agent we need to add conversational memory to make the agent remember previous interactions. We'll be using the older `ConversationBufferMemory` class rather than the newer `RunnableWithMessageHistory` — the reason being that we will also be using the older `create_tool_calling_agent` and `AgentExecutor` method and class.

In the `05` chapter we will be using the newer `RunnableWithMessageHistory` class as we'll be building a custom `AgentExecutor`.

In [2]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",  # must align with MessagesPlaceholder variable_name
    return_messages=True  # to return Message objects
)

<ipython-input-2-07edbbd232c1>:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


Now we will initialize our agent. For that we need:

* `llm`: as already defined
* `tools`: to be defined (just a list of our previously defined tools)
* `prompt`: as already defined
* `memory`: as already defined

In [9]:
from langchain.agents import create_tool_calling_agent

tools = [add, subtract, multiply, exponentiate]

agent = create_tool_calling_agent(
    llm=llm, tools=tools, prompt=prompt
)

Our `agent` by itself is like one-step of our agent execution loop. So, if we call the `agent.invoke` method it will get the LLM to generate a single response and go no further, so no tools will be executed, and no next iterations will be performed.

We can see this by asking a query that should trigger a tool call:

In [11]:
agent.invoke({
    "input": "what is 10.7 multiplied by 7.68?",
    "chat_history": memory.chat_memory.messages,
    "intermediate_steps": []  # agent will append it's internal steps here
})

AgentFinish(return_values={'output': '<multiply>{"x": 10.7, "y": 7.68}</multiply>'}, log='<multiply>{"x": 10.7, "y": 7.68}</multiply>')

Here, we can see the LLM has generated that we should use the `multiply` tool and the tool input should be `{"x": 10.7, "y": 7.68}`. However, the tool is not executed. For that to happen we need an agent execution loop, which will handle the multiple iterations of generation to tool calling to generation, etc.

We use the `AgentExecutor` class to handle the execution loop:

In [12]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=True
)

Now let's try the same query with the executor, note that the `intermediate_steps` parameter that we added before is no longer needed as the executor handles it internally.

In [13]:
agent_executor.invoke({
    "input": "what is 10.7 multiplied by 7.68?",
    "chat_history": memory.chat_memory.messages,
})

<multiply>{"x": 10.7, "y": 7.68}</multiply>

> Finished chain.


{'input': 'what is 10.7 multiplied by 7.68?',
 'chat_history': [HumanMessage(content='what is 10.7 multiplied by 7.68?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='<multiply>{"x": 10.7, "y": 7.68}</multiply>', additional_kwargs={}, response_metadata={})],
 'output': '<multiply>{"x": 10.7, "y": 7.68}</multiply>'}

We can see that the `multiply` tool was invoked, producing the observation of `82.175999...`. After the observation was provided, we can see that the LLM then generated a final response of:

```
10.7 multiplied by 7.68 is approximately 82.18.
```

This final response was generated based on the original query and the tool output (ie the _observation_). We can also confirm that this answer is accurate:

In [14]:
10.7*7.68

82.17599999999999

Let's test our agent with some memory and tool use. First, we tell it our name, then we will perform a few tool calls, then see if the agent can still recall our name.

First, give the agent our name:

In [15]:
agent_executor.invoke({
    "input": "My name is James",
    "chat_history": memory
})

That's nice to know, but it doesn't seem to be related to the previous math problem. If you'd like to continue with the math problem, I can help you with that.

> Finished chain.


{'input': 'My name is James',
 'chat_history': [HumanMessage(content='what is 10.7 multiplied by 7.68?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='<multiply>{"x": 10.7, "y": 7.68}</multiply>', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is James', additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's nice to know, but it doesn't seem to be related to the previous math problem. If you'd like to continue with the math problem, I can help you with that.", additional_kwargs={}, response_metadata={})],
 'output': "That's nice to know, but it doesn't seem to be related to the previous math problem. If you'd like to continue with the math problem, I can help you with that."}

Now let's try and get the agent to perform multiple tool calls within a single execution loop:

In [16]:
agent_executor.invoke({
    "input": "What is nine plus 10, minus 4 * 2, to the power of 3",
    "chat_history": memory
})


Invoking: `add` with `{'x': 9, 'y': 10}`


19.0
Invoking: `multiply` with `{'x': 4, 'y': 2}`


8.0
Invoking: `exponentiate` with `{'x': 8, 'y': 3}`


512.0
Invoking: `subtract` with `{'x': 512, 'y': 19}`


-493.0The result is -493.

> Finished chain.


{'input': 'What is nine plus 10, minus 4 * 2, to the power of 3',
 'chat_history': [HumanMessage(content='what is 10.7 multiplied by 7.68?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='<multiply>{"x": 10.7, "y": 7.68}</multiply>', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is James', additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's nice to know, but it doesn't seem to be related to the previous math problem. If you'd like to continue with the math problem, I can help you with that.", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is nine plus 10, minus 4 * 2, to the power of 3', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The result is -493.', additional_kwargs={}, response_metadata={})],
 'output': 'The result is -493.'}

Let's confirm that the answer is accurate:

In [17]:
9+10-(4*2)**3

-493

Perfect, now let's see if the agent can still recall our name:

In [18]:
agent_executor.invoke({
    "input": "What is my name",
    "chat_history": memory
})

James

> Finished chain.


{'input': 'What is my name',
 'chat_history': [HumanMessage(content='what is 10.7 multiplied by 7.68?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='<multiply>{"x": 10.7, "y": 7.68}</multiply>', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is James', additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's nice to know, but it doesn't seem to be related to the previous math problem. If you'd like to continue with the math problem, I can help you with that.", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is nine plus 10, minus 4 * 2, to the power of 3', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The result is -493.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is my name', additional_kwargs={}, response_metadata={}),
  AIMessage(content='James', additional_kwargs={}, response_metadata={})],
 'output': 'James'}

The agent has successfully recalled our name. Let's move on to another agent example.

## SerpAPI Weather Agent

In this example, we'll be using the same agent and executor setup as before, but we'll be adding the [SerpAPI](https://serpapi.com/users/sign_in) service to allow our agent to search the web for information.

To use this tool, you need an API key, with the free plan you can use up to 100 searches per month.

In [ ]:
os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY") \
    or getpass("Enter your SerpAPI API key: ")

Enter your SerpAPI API key: ··········


Here we will load the `serpapi` tool directly from the prebuilt tools that LangChain provides.

In [23]:
from langchain.agents import load_tools

toolbox = load_tools(tool_names=[], llm=llm)

These custom tools can look into your IP address, find out where you are currently, then we will also use a secondary function to get the current date and time, then we will use this information to feed into the SerpAPI to find us the weather pattern in your area and at the time of the function calling.

In [19]:
import requests
from datetime import datetime

@tool
def get_location_from_ip():
    """Get the geographical location based on the IP address."""
    try:
        response = requests.get("https://ipinfo.io/json")
        data = response.json()
        if 'loc' in data:
            latitude, longitude = data['loc'].split(',')
            data = (
                f"Latitude: {latitude},\n"
                f"Longitude: {longitude},\n"
                f"City: {data.get('city', 'N/A')},\n"
                f"Country: {data.get('country', 'N/A')}"
            )
            return data
        else:
            return "Location could not be determined."
    except Exception as e:
        return f"Error occurred: {e}"

@tool
def get_current_datetime() -> str:
    """Return the current date and time."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

We can create our prompt, this time we'll skip the `chat_history` part as we don't need it. However, you can add it if preferred.

In [20]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

Now we create our full `tools` list, our `agent`, and the `agent_executor`:

In [24]:
tools = toolbox + [get_current_datetime, get_location_from_ip]

agent = create_tool_calling_agent(
    llm=llm, tools=tools, prompt=prompt
)

agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True
)

For me I have to specify to the AI as I live in the UK and alot of places in the UK also exist in USA, which is why I explicitly state to use the country in the search as well as the town / city.

In [25]:
out = agent_executor.invoke({
    "input": (
        "I have a few questions, what is the date and time right now? "
        "How is the weather where I am? Please give me degrees in Celsius"
    )
})


Invoking: `get_current_datetime` with `{}`


2025-05-28 11:40:51
Invoking: `get_location_from_ip` with `{}`
responded: Unfortunately, I can't access your location to provide the weather information. However, I can suggest a way to get the current weather in your location. We can use the 'get_location_from_ip' function to get your location and then use that information to get the current weather.

Here's how you can do it:



Latitude: 41.2619,
Longitude: -95.8608,
City: Council Bluffs,
Country: USUnfortunately, I cannot find any information about the current weather in Council Bluffs, US.

> Finished chain.


In [26]:
from IPython.display import display, Markdown

display(Markdown(out["output"]))

Unfortunately, I cannot find any information about the current weather in Council Bluffs, US.

That's the correct answer, and we even get the approximate answer in Celsius despite the tool returning the temperature in Fahrenheit.

We've finished our into to LangChain Agents, in the next chapter we will be looking at how to create custom agents and executors.

---